**import dataset**

In [1]:
!git clone https://github.com/elaamranisoufiane/BNS_1024_test.git

Cloning into 'BNS_1024_test'...
remote: Enumerating objects: 164, done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 164
Receiving objects: 100% (164/164), 111.35 MiB | 24.28 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (612/612), done.


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

 

**performance metrics**


In [6]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

**Our Model**


In [7]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    s = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(s)
    return s, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    """ Input layer """
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 32)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 128)
    s4, p4 = encoder_block(p3, 256)
    #add layer
    s5, p5 = encoder_block(p4, 512)
    

    """ Bottleneck """
    b1 = conv_block(p5, 1024)

    """ Decoder """
    #add layer
    d0 = decoder_block(b1, s5, 512)
    
    d1 = decoder_block(d0, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    """ Output layer """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
    model = build_unet((256, 256, 3))
    model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 256, 256, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 256, 256, 32) 128         conv2d_23[0][0]                  
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 256, 256, 32) 0           batch_normalization_22[0][0]     
_______________________________________________________________________________________________

In [11]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from sklearn.model_selection import KFold
from tqdm import tqdm
import pandas as pd

#import build_unet
#import dice_coef, iou
 
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


# input image size
H = 1024
W = 1024


# create directory and gives the path
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x


def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x


def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y



def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

##########################################

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image1(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  ## (1, 256, 256, 3)
    return ori_x, x


def read_mask1(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)

def load_data(path, split=0.49):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    size = int(len(images) * split)

    train_x = images
    train_y =masks
    valid_x=train_x
    valid_y=train_y


    test_x= train_x
    test_y=train_y
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

# * # **training part**

In [ ]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)
    # create a file in the same ocation
    create_dir('files')
    """ Hyperparaqmeters """
    batch_size = 8
    lr = 1e-4  ## 0.0001
    num_epochs = 260
    model_path = "files/model.h5"
    csv_path = "files/data.csv"
    create_dir("results")
    """ Dataset """
    #dataset_path = "./DBA"
    # colab
    # dataset_path = "/content/U-Net-for-Nuclei-Semantic-Segmentation/DSB"
    # kuggle
    dataset_path = "./2018CompDatasetNormClean"
    ##########################
    
    # Define per-fold score container
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    '''
    train_x = pd.DataFrame(train_x)
    test_x = pd.DataFrame(test_x)
    train_x = pd.concat([train_x, test_x]) 
    
    train_y = pd.DataFrame(train_y)
    test_y = pd.DataFrame(test_y)
    train_y = pd.concat([train_y, test_y]) 
    '''
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")
    
    acc_per_fold = []
    loss_per_fold = []
    acc_per_fold = []
    f1_value_per_fold = []
    jac_value_per_fold = []
    recall_value_per_fold = []
    precision_value_per_fold = []
  
    
    num_folds=4
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=num_folds, shuffle=True)
    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(train_x, train_y):
        print(f'Training for fold {fold_no} ...')
        test_x1=[]
        test_y1=[]
        for test_i in test:
            test_x1.append(train_x[test_i])
            test_y1.append(train_y[test_i])
        train_x1=[]
        train_y1=[]
        
        for train_i in train:
            train_x1.append(train_x[train_i])
            train_y1.append(train_y[train_i])
        #fold_no = fold_no + 1
        
        
        #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
        #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path) 
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
        #train_x1, train_y1 = shuffle(train_x1, train_y1)
        print(f"Train: {len(train_x1)} - {len(train_y1)}")
        print(f"Valid: {len(valid_x)} - {len(valid_y)}")
        print(f"Test: {len(test_x1)} - {len(test_y1)}")
        train_dataset = tf_dataset(train_x1, train_y1, batch=batch_size)
        valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
        train_steps = (len(train_x1) // batch_size)
        valid_steps = (len(valid_x) // batch_size)
        if len(train_x1) % batch_size != 0:
            train_steps += 1
        #if len(valid_x) % batch_size != 0:
            #valid_steps += 1

        for x, y in valid_dataset:
            print(x.shape, y.shape)
            break
        """ build the model with U-Net network archtecture """
        model = build_unet((H, W, 3))
        metrics = ["accuracy", dice_coef, iou, Recall(), Precision()]
        model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
        callbacks = [
            ModelCheckpoint(model_path, verbose=1, save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
            CSVLogger(csv_path),
            # EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
        ]
        history = model.fit(
            train_dataset,
            epochs=num_epochs,
            validation_data=valid_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            callbacks=callbacks,
            shuffle=False
        )
        fold_no = fold_no + 1
        ### evaluation
        #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
        """ Load Model """
        with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
            model = tf.keras.models.load_model(model_path)
            #model.summary()
        """ Prediction and metrics values """
        SCORE = []
        print(f'test image : {test_x1[1]}')
        for x, y in tqdm(zip(test_x1, test_y1), total=len(test_x1)):
            name = x.split("/")[-1]
            
            """ Reading the image and mask """
            ori_x, x = read_image1(x)
            ori_y, y = read_mask1(y)
            """ Prediction """
            y_pred = model.predict(x)[0] > 0.5
            y_pred = np.squeeze(y_pred, axis=-1)
            y_pred = y_pred.astype(np.int32)
            
            save_path = f"results/{name}"
            save_result(ori_x, ori_y, y_pred, save_path)
            
            """ Flattening the numpy arrays. """
            y = y.flatten()
            y_pred = y_pred.flatten()
            
            """ Calculating metrics values """
            acc_value = accuracy_score(y, y_pred)
            f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
            jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
            recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
            precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
            SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])
           
    
        
            """ Metrics values """
        print("eval metrics")
        score = [s[1:] for s in SCORE]
        score = np.mean(score, axis=0)
        print(f"Accuracy: {score[0]:0.5f}")
        print(f"F1: {score[1]:0.5f}")
        print(f"Jaccard: {score[2]:0.5f}")
        print(f"Recall: {score[3]:0.5f}")
        print(f"Precision: {score[4]:0.5f}")
        
        """ save the results  """
        acc_per_fold.append(score[0])
        f1_value_per_fold.append(score[1])
        jac_value_per_fold.append(score[2])
        recall_value_per_fold.append(score[3])
        precision_value_per_fold.append(score[4])
           

        """ Saving all the results """
        df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
        df.to_csv("files/score.csv")

""" final metrics"""

print(f"Accuracy: {acc_per_fold}")
print(f"F1: {f1_value_per_fold}")
print(f"Jaccard: {jac_value_per_fold}")
print(f"Recall: {recall_value_per_fold}")
print(f"Precision: {precision_value_per_fold}")
        
    ##########################

In [ ]:
print(f"Accuracy: {np.ceil(np.mean(acc_per_fold)*10000)/100}%")
print(f"F1: {np.ceil(np.mean(f1_value_per_fold)*10000)/100}%")
print(f"Jaccard: {np.ceil(np.mean(jac_value_per_fold)*10000)/100}%")
print(f"Recall: {np.ceil(np.mean(recall_value_per_fold)*10000)/100}%")
print(f"Precision: {np.ceil(np.mean(precision_value_per_fold)*10000)/100}%")

In [ ]:
    '''
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss.png')
    plt.show()
    #plt.imshow(mpimg.imread('./loss.png'))

    accu = history.history['accuracy']
    val_accu = history.history['val_accuracy']
    epochs = range(1, len(accu) + 1)
    plt.plot(epochs, accu, 'r', label='accuracy') 
    plt.plot(epochs, val_accu, 'b', label='val_accuracy') 
    plt.title('accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig('accuracy.png')
    plt.show()
    #plt.imshow(mpimg.imread('./accuracy.png'))
    '''

# # testing part****

In [12]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
#import dice_coef, iou
#import load_data

H = 1024
W = 1024


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  ## (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x


def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)
if __name__ == "__main__":
    create_dir("results2")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("/kaggle/input/isr-model1/model10.h5")
        #model.summary()

    """ Dataset """
    dataset_path = "/kaggle/working/BNS_1024_test"
    #dataset_path = "/imagesToAugment/images"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Prediction and metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]

        """ Reading the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"results2/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays. """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

 

100%|██████████| 300/300 [06:53<00:00,  1.38s/it]

Accuracy: 0.90926
F1: 0.23084
Jaccard: 0.14107
Recall: 0.16279
Precision: 0.72053


##################################""

##################################""

In [ ]:
'''
import cv2
import glob
import matplotlib.pyplot as plt
    
path="./results/*.jpg"

images=[cv2.imread(image) for image in glob.glob(path)]
fig=plt.figure()
for i in range(len(images)):
    #plt.subplot(5,5,i+1)
    plt.imshow(images[i])
    plt.show()'''

In [ ]:
'''!zip -r ./file.zip ./results '''

In [ ]:
'''from IPython.display import FileLink
FileLink(r'file.zip')'''